LAB 2: Signal to Noise Ratio, Quantization
1. SIGNAL TO NOISE RATIO (40%)
Generate a tone with frequency 2 MHz and amplitude 1 V. Sample the
tone at frequencies Fs = 5 MHz.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

Fs = 5e6  # Sampling frequency
N = 1024  # Number of samples
SNR_dB = 50

t = np.arange(N) / Fs
x_clean = np.cos(2 * np.pi * 2e6 * t)

a) Add Gaussian noise to the sampled sinewave such that the signal SNR is
50 dB. Find first the variance of the Gaussian noise needed to produce the
target SNR. Calculate and plot the Power Spectral Density (PSD) from the DFT of the noisy samples. Corroborate that the SNR calculation from the DFT plot gives the theoretical result. What would be the variance of a uniformly distributed noise to obtain the same SNR.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

Fs = 5e6  # Sampling frequency
N = 1024  # Number of samples
SNR_dB = 50

t = np.arange(N) / Fs
x_clean = np.cos(2 * np.pi * 2e6 * t)

Psig = (1*1)/2  # sine wave power is half
Pn = Psig/ 10**(SNR_dB/10)
# variance is sqrt of noise power
noise_std = np.sqrt(Pn)

# Gaussian noise
noise = np.random.normal(0, noise_std, N)
x = x_clean + noise

X = np.fft.fft(x)
freqs = np.fft.fftfreq(N, d=1/Fs)
# Normalized PSD
PSD = (np.abs(X) ** 2) / (N * Fs)  

plt.figure(figsize=(8, 5))
plt.plot(freqs[:N//2], 2*PSD[:N//2])
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD (Watt/Hz)')
plt.title('Power Spectral Density')
plt.grid()
plt.show()

# Find peak
signal_bin = np.argmax(PSD)
Psig_sim = PSD[signal_bin]
noise_floor = np.mean(PSD[np.r_[0:signal_bin-10, signal_bin+10:]])
SNR_dB_sim = 10 * np.log10(Psig_sim / noise_floor)
print(f"Simulated SNR from PSD: {SNR_dB_sim:.2f} dB")

# Compute uniform noise variance for the same SNR
# uniform noise variance noise_uni = A^2/3, Pnoise_uni=A^2=3*noise_uni=noise_std

# Print results
print(f"Theoretical Noise Variance (Gaussian): {noise_std:.2e}")
print(f"Estimated SNR from PSD: {SNR_dB_sim:.2f} dB")
print(f"Variance of Uniform Noise for the Same SNR: {noise_std/3:.2e}")

NameError: name 'SNR_dB' is not defined

b) Now repeat a.) using a window before the DFT. Use the following
windows: Hanning, Hamming, Blackman. What are your conclusions?
NOTE: The use of windows mentioned above spreads the signal power. You
must take this into account when computing SNR.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch

Fs = 5e6  # Sampling frequency
N = 1024  # Number of samples
SNR_dB = 50

t = np.arange(N) / Fs
x_clean = np.cos(2 * np.pi * 2e6 * t)

Psig = (1*1)/2  # sine wave power is half
Pn = Psig/ 10**(SNR_dB/10)
# variance is sqrt of noise power
noise_std = np.sqrt(Pn)

# Gaussian noise
noise = np.random.normal(0, noise_std, N)
x = x_clean + noise

# Define windows
windows = {
    "Hanning": np.hanning(N),
    "Hamming": np.hamming(N),
    "Blackman": np.blackman(N)
}

def compute_snr(psd, freqs, signal_freq, fs):
    signal_bin = np.argmax(psd[:N//2])  # Find peak in first half
    signal_power_est = np.sum(psd[signal_bin-1:signal_bin+2])  # Sum main lobe bins
    
    # Estimate noise power (excluding main lobe)
    noise_power_est = np.sum(psd[:N//2]) - signal_power_est
    
    # Compute SNR
    return 10 * np.log10(signal_power_est / noise_power_est)

# Plot results for each window
plt.figure(figsize=(10, 6))

for name, window in windows.items():
    windowed_signal = x * window

    # Compute coherent gain and correct power
    G = np.sum(window) / N
    power_correction = 1 / (G ** 2)

    fft_signal = np.fft.fft(windowed_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs) * power_correction  # Corrected PSD

    freqs = np.fft.fftfreq(N, d=1/fs)

    # Compute SNR from PSD
    snr_estimated = compute_snr(psd, freqs, 2e6, Fs)
    print(f"Estimated SNR from PSD: {snr_estimated:.2f} dB")

    # Plot PSD
    plt.plot(freqs[:N//2], 10 * np.log10(psd[:N//2]), label=f"{name} Window (SNR: {snr_estimated:.2f} dB)")

plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (dB/Hz)")
plt.title("Power Spectral Density with Different Windows")
plt.legend()
plt.grid()
plt.show()


2. QUANTIZATION (60%)
a) Create a perfect quantizer with 6 bits of resolution and with flexible sampling rate. For a 200 MHz full scale input tone, sample and quantize the sinewave at 400 MHz and plot the PSD of 30 periods. What is the SNR? Repeat the SNR calculation for 100 periods of the same signal. Make your own conclusions about this test regarding periodicity of quantization noise and the impact of this in the SNR. How can you solve this problem?

ANS:

SNR for an ideal quantizer is SNR=6.02N+1.76
for N_bits=6, SNR=37.98dB

When sample clock frequency is exactly twice of the signal frequency, 400MHz, the signal cannot be accurately recovered.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fs = 400e6
f_signal = 200e6
N_bits = 6
full_scale = 1.0

def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t)

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

# Test Case 1: 30 periods
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 30)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)

# Test Case 2: 100 periods
t2, s2, q2, N2 = quantize_signal(f_signal, fs, 100)
freqs2, psd2, SNR2 = compute_psd_and_snr(q2, N2, fs, f_signal)

# Print SNR results
print(f"SNR with 30 periods: {SNR1:.2f} dB")
print(f"SNR with 100 periods: {SNR2:.2f} dB")

# Plot PSD results
plt.figure(figsize=(10,6))
plt.plot(freqs1[:N1//2], 10 * np.log10(psd1[:N1//2]), label=f"30 Periods (SNR: {SNR1:.2f} dB)")
plt.plot(freqs2[:N2//2], 10 * np.log10(psd2[:N2//2]), label=f"100 Periods (SNR: {SNR2:.2f} dB)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (dB/Hz)")
plt.title("PSD of Quantized Signal for Different Window Sizes")
plt.legend()
plt.grid()
plt.show()


b) Find an incommensurate sampling frequency larger than Nyquist rate.
Plot the PSD of the new samples. Calculate the SNR from the figure.

Use sample clock 450MHz. PSD can be plotted and signal can be recovered.
Once the number of signal periods increase, we effectively increase the total number of samples (N) in DFT window. The number of DFT points (N) increases which result in more frequency bins. Since the total noise power remains constant, it gets spread over more bins and reduce power in each bin: Noise Power Per Bin ∝ 1/N. ince signal power remains concentrated in a few bins, but the noise spreads, SNR will improve and is closer to ideal SNR as consequence. Also more number of signal periods can make qunantization noise periodic which shows as spectral line in DFT.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fs = 450e6
f_signal = 200e6
N_bits = 6
full_scale = 1.0

def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t + 0.25*np.pi)

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

# Test Case 1: 30 periods
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 30)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)

# Test Case 2: 100 periods
t2, s2, q2, N2 = quantize_signal(f_signal, fs, 100)
freqs2, psd2, SNR2 = compute_psd_and_snr(q2, N2, fs, f_signal)

# Print SNR results
print(f"SNR with 30 periods: {SNR1:.2f} dB")
print(f"SNR with 100 periods: {SNR2:.2f} dB")

# Plot PSD results
plt.figure(figsize=(10,6))
plt.plot(freqs1[:N1//2], 10 * np.log10(psd1[:N1//2]), label=f"30 Periods (SNR: {SNR1:.2f} dB)")
plt.plot(freqs2[:N2//2], 10 * np.log10(psd2[:N2//2]), label=f"100 Periods (SNR: {SNR2:.2f} dB)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (dB/Hz)")
plt.title("PSD of Quantized Signal for Different Window Sizes")
plt.legend()
plt.grid()
plt.show()

c) Repeat a) using a 12 bit quantizer. Can you prove from simulations that
SNR ~ 6N (where N is the number of bits used by the quantizer) in both the
cases, N = 6 and N = 12?

For 6-bit ADC, SNR simulated is 45.91dB. 6*N=36dB
For 12-bit ADC, SNR simulated is 73.32dB, 6*N=72dB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t + 0.25*np.pi)

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

fs = 450e6
f_signal = 200e6
full_scale = 1.0

N_bits = 6
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 6-bit ADC: {SNR1:.2f} dB")

N_bits = 12
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 12-bit ADC: {SNR1:.2f} dB")

d) Use a Hanning window and repeat c). What is the SNR? Make your own
conclusions.

SNR for 6-bit ADC: 34.36 dB
SNR for 12-bit ADC: 54.50 dB
Hanning window makes 6-bit ADC SNR closer to quantization SNR (6N). But SNR has been reduced due to loss of signal power after going through filter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t)
    signal = signal * np.hanning(N)

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

fs = 450e6
f_signal = 200e6
full_scale = 1.0

N_bits = 6
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 6-bit ADC: {SNR1:.2f} dB")

N_bits = 12
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 12-bit ADC: {SNR1:.2f} dB")

e) Now add noise again so the signal SNR is 38 dB . Repeat c) and d). What
are the SNRs? Provide conclusions.

w/o hanning window:
SNR for 6-bit ADC: 34.47 dB
SNR for 12-bit ADC: 37.91 dB
After adding noise with SNR=38dB, the result SNR of ADC is minimum of signal SNR and ADC quantization SNR:

with Hanning window:
SNR for 6-bit ADC: 32.59 dB
SNR for 12-bit ADC: 38.40 dB
Hanning window further reduces SNR since Hanning window can reduce signal power by around 1.5dB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# w/o hanning window
def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t)
    
    Psig = (full_scale*full_scale)/2  # sine wave power is half
    SNR_dB = 38
    Pn = Psig/ 10**(SNR_dB/10)
    # variance is sqrt of noise power
    noise_std = np.sqrt(Pn)
    noise = np.random.normal(0, noise_std, N)
    signal = signal + noise

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

fs = 450e6
f_signal = 200e6
full_scale = 1.0

N_bits = 6
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 6-bit ADC: {SNR1:.2f} dB")

N_bits = 12
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 12-bit ADC: {SNR1:.2f} dB")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# w/o hanning window
def quantize_signal(f_signal, fs, num_periods):
    # Number of samples
    N = int(num_periods * (fs / f_signal))
    t = np.arange(N) / fs
    signal = full_scale * np.sin(2 * np.pi * f_signal * t)
    
    Psig = (full_scale*full_scale)/2  # sine wave power is half
    SNR_dB = 38
    Pn = Psig/ 10**(SNR_dB/10)
    # variance is sqrt of noise power
    noise_std = np.sqrt(Pn)
    noise = np.random.normal(0, noise_std, N)
    signal = signal + noise
    signal = signal * np.hanning(N)

    # Quantization process
    levels = 2 ** N_bits
    delta = (2 * full_scale) / levels
    quantized_signal = np.round(signal / delta) * delta

    return t, signal, quantized_signal, N

# Function to compute PSD and SNR
def compute_psd_and_snr(quantized_signal, N, fs, f_signal):
    fft_signal = np.fft.fft(quantized_signal, N)
    psd = (np.abs(fft_signal) ** 2) / (N * fs)

    freqs = np.fft.fftfreq(N, d=1/fs)
    
    # Identify the signal power (main tone at f_signal)
    signal_bin = np.argmax(psd[:N//2])
    # Sum main lobe bins
    signal_power = np.sum(psd[signal_bin-1:signal_bin+2])  
    
    # Estimate noise power by excluding signal bins
    noise_power = np.sum(psd[:N//2]) - signal_power
    
    # Compute SNR
    SNR = 10 * np.log10(signal_power / noise_power)
    return freqs, psd, SNR

fs = 450e6
f_signal = 200e6
full_scale = 1.0

N_bits = 6
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 6-bit ADC: {SNR1:.2f} dB")

N_bits = 12
t1, s1, q1, N1 = quantize_signal(f_signal, fs, 100)
freqs1, psd1, SNR1 = compute_psd_and_snr(q1, N1, fs, f_signal)
print(f"SNR for 12-bit ADC: {SNR1:.2f} dB")